In [2]:
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [3]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [4]:
with open('password.txt') as f:
    password = f.read()

In [5]:
email = 'jhsuslovemy@naver.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
# 로그인 정보 - 나중에 저장
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [8]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 2. 제주도 맛집으로 검색

In [ ]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

In [ ]:
driver.get(url)
time.sleep(5)

## 3. 첫번째 게시글 열기

In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
import unicodedata

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

In [ ]:
# 문자에서 #을 이용해 해시태그를 분류해내기.. '#[^\s#, \\]+'

import re
tags = re.findall(r'#[^\s#, \\]+', content)
tags

In [ ]:
# 작성일자 정보 가져오기 _1o9PC Nzb55
date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]
date

In [ ]:
# 좋아요 수 가져오기 
# print(soup.select_one('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > a > span').text)
try:
    # like = soup.select_one('div.Nm9Fw.zV_NJ').text
    # like = soup.select_one('/html/body/div[6]/div[2]/div/article/div[3]/section[2]/div/div/a/span').text
    like = int(soup.select_one('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > a > span').text.replace(',',''))
except:
    like = 0
like

In [ ]:
# 위치정보 가져오기
try:
    place = soup.select('div._M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

In [ ]:
row = [content, date, like, place, tags]
row

## 5. 다음 게시글로 이동

In [ ]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(1)

In [ ]:
# 열린 창 닫기 body > div._2dDPU.CkGkG > div.Igw0E.IwRSH.eGOV_._4EzTm.BI4qX.qJPeX.fm1AK.TxciK.yiMZG > button
# /html/body/div[6]/div[3]/button
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

## 6. 다중으로 크롤링하기

In [ ]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
results = []

for i in range(50):
    print(i, end=' ')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#, \\]+', content)
    date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]

    try:
    # like = soup.select_one('div.Nm9Fw.zV_NJ').text
    # like = soup.select_one('/html/body/div[6]/div[2]/div/article/div[3]/section[2]/div/div/a/span').text
        # like = int(soup.select_one('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > a > span').text.replace(',',''))
        like = int(soup.select_one('div.Nm9Fw').text[4:-1].replace(',',''))
    except:
        like = 0

    try:
        place = soup.select('div._M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    row = [content, date, like, place, tags]
    results.append(row)

    driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
    time.sleep(2)

In [ ]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

## 500개의 데이터를 가져오기.

## - 제주도 맛집, 제주맛집, 제주도 관광, 제주여행

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
# from tqdm.notebook import tqdm
from urllib.parse import quote
import unicodedata
import re
import time
import pandas as pd

In [ ]:
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [13]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#, \\]+', content)
    date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]

    try:
        like = int(soup.select_one('div.Nm9Fw').text[4:-1].replace(',',''))
    except:
        like = 0

    try:
        place = soup.select('div._M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]

In [14]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
results = []

for _ in range(500):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

In [16]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(3)

    results = []

    for _ in range(200):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

    df = pd.DataFrame(results, columns=['content', 'tags', 'date', 'like', 'place'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(5)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a._65Bje.coreSpriteRightPaginationArrow"}
  (Session info: chrome=92.0.4515.159)
